# MNIST Handwritten Digits Classification Problem

## Introduction

This notebook serves to detail a report along with documented code on the efficacy of a neural network in classifying handwritten digits. It will demonstrate the thought process used in the problem, as well as the underlying code sectioned into modules.

## Table of Contents

1. [Problem Statement](#ps)
2. [Dataset](#d)
3. [Theoretical Implementation](#ti)
4. [Code Implementation](#ci)
5. [Testing](#te)

## Problem Statement  <a id='ps'></a>

The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

The idea is to develop a classification algorithm that can determine what numerical value the handwritten digits correspond to. We will attempt to solve this by developing a neural network. This sort of classification is common among banks in recognizing handwritten cheques, and can be used in numerous applications elsewhere.

## Dataset <a id='d'></a>

The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image by computing the center of mass of the pixels, and translating the image so as to position this point at the center of the 28x28 field. 

The data is stored in a very simple file format designed for storing vectors and multidimensional matrices. 
All the integers in the files are stored in the MSB first (high endian) format used by most non-Intel processors. Users of Intel processors and other low-endian machines must flip the bytes of the header.

There are 4 files:

1. train-images-idx3-ubyte: training set images 

2. train-labels-idx1-ubyte: training set labels 

3. t10k-images-idx3-ubyte:  test set images 

4. t10k-labels-idx1-ubyte:  test set labels

The training set contains 60000 examples, and the test set 10000 examples.

The first 5000 examples of the test set are taken from the original NIST training set. The last 5000 are taken from the original NIST test set. The first 5000 are cleaner and easier than the last 5000.

#### TRAINING SET LABEL FILE (train-labels-idx1-ubyte):
        
    offset   type            value            description
    0000     32 bit integer  0x00000801(2049) magic number (MSB first) 
    0004     32 bit integer  60000            number of items 
    0008     unsigned byte   ??               label 
    0009     unsigned byte   ??               label 
    ........ 
    xxxx     unsigned byte   ??               label
    The labels values are 0 to 9.

#### TRAINING SET IMAGE FILE (train-images-idx3-ubyte):

    [offset] [type]          [value]          [description] 
    0000     32 bit integer  0x00000803(2051) magic number 
    0004     32 bit integer  60000            number of images 
    0008     32 bit integer  28               number of rows 
    0012     32 bit integer  28               number of columns 
    0016     unsigned byte   ??               pixel 
    0017     unsigned byte   ??               pixel 
    ........ 
    xxxx     unsigned byte   ??               pixel
    
Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black).

#### TEST SET LABEL FILE (t10k-labels-idx1-ubyte):

    [offset] [type]          [value]          [description] 
    0000     32 bit integer  0x00000801(2049) magic number (MSB first) 
    0004     32 bit integer  10000            number of items 
    0008     unsigned byte   ??               label 
    0009     unsigned byte   ??               label 
    ........ 
    xxxx     unsigned byte   ??               label
   
The labels values are 0 to 9.

#### TEST SET IMAGE FILE (t10k-images-idx3-ubyte):

    [offset] [type]          [value]          [description] 
    0000     32 bit integer  0x00000803(2051) magic number 
    0004     32 bit integer  10000            number of images 
    0008     32 bit integer  28               number of rows 
    0012     32 bit integer  28               number of columns 
    0016     unsigned byte   ??               pixel 
    0017     unsigned byte   ??               pixel 
    ........ 
    xxxx     unsigned byte   ??               pixel
        
Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black). 

We will split the data in the training set further to obtain a *training* and *validation* set. Therefore, we are looking at
1. 50,000 example training set
2. 10,000 example validation set
3. 10,000 example test set

To load the dataset, we will need to write an appropriate program. 

In [33]:
"""
mnist_loader  :   ADAPTED FROM WEBSITE
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import cPickle
import gzip
import os

# Third-party libraries
import numpy as np

def load_data():
    
    """
    Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    
    """
    
    f = gzip.open(os.path.expanduser("~/Documents/Data Science Projects/MNIST/mnist.pkl.gz"), 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    
    """
    Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code.
    
    """
    
    
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    
    """
    Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network.
    
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

## Theoretical Implementation  <a id='ti'></a>

### Segmentation Problem

First, we may need to derive some elementary code to segment the images such that we can work with one digit per classification.

### Classification Problem

As is probably obvious from the description of the problem, we are looking for a classification of numerical values 0-9. This means we can think of our neural net as having 10 outputs, one for each numerical value. Since the images are 28x28 pixels, we can think of the input of our neural net as having 28 squared, or 784 inputs. We will now try to derive a method to determine the weights and biases of the neural net. 

#### Cost Function

In deciding what the weights and biases ($w$ and $b$) of the neural network should be, it becomes necessary to define a cost function. We can think of our cost function as the following, where $x$ is the input image, $y$ is the desired output, $a$ is the vector of outputs from the network given $x$, $n$ is the total number of training examples, $w$ the collection of weights and $b$ the biases. The cost function becomes 

\begin{align}
C(w,b) = \frac{1}{2n}\sum_x^n ||y - a(x)||^2
\end{align}

We implement a quadratic cost function for smoothness in response to changes in $w$ and $b$.

In order to minimize the function, we have various tools at our disposal. We could implement a convex optimization package or other minimization tools, but for this example we will use the simplest to implement: a gradient descent.

### Code Implementation <a id='ci'></a>

This will become our Network Module.

In [34]:
#Libraries
import numpy as np
import random


class Network(object):

    def __init__(self, sizes):
        """
        init
        :param sizes: array containing sizes of layers
        :type sizes: list, int
        
        """
        assert isinstance(sizes,list)
        assert all(isinstance(a,int) for a in sizes)
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        #initialize with random biases and weights
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """
        Return the output of the network if "a" is input.
        
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        Train the neural network using mini-batch stochastic
        gradient descent.  
        :param training_data: list of tuples containing input and desired output
        :type training_data: list, tuple, int
        :param epochs: no. of epochs to train for
        :type epochs: int
        :param mini_batch_size: size of batch
        :type mini_batch_size: int
        :param eta: learning rate
        :type eta: float
        :param test_data: list of tuples containing test data input, desired output
        :type test_data: list, tuple, int
        
        """
        if test_data: 
            n_test = len(test_data)
            
        #size of training set
        
        n = len(training_data)
        
        #for each epoch
        
        for j in xrange(epochs):
            
            #shuffle training data
            
            random.shuffle(training_data)
            
            #create batches
            
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            
            #use update function for each batch
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            #evaluate learning algorithm if test data is supplied
            
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
                
    def update_mini_batch(self, mini_batch, eta):
        """
        Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        
        :param mini_batch: list of tuples containing input and desired output
        :type mini_batch: list, tuples, int
        :param eta: learning rate
        :type eta: float
        
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            
            #back propagate gradient descent
            
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        #update weights and biases
        
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        """
        Return a tuple (nabla_b, nabla_w) representing the
        gradient for the cost function C_x.  nabla_b and
        nabla_w are layer-by-layer lists of numpy arrays, similar
        to self.biases and self.weights.
        
        :param x: 
        :type x:
        :param y:
        :type y:
        
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    
    def evaluate(self, test_data):
        """
        Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation.
        :param test_data: list of test data labels
        :type test_data: list, int
        
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations.
        :param output_activations:
        :type output_activations: 
        :param y:
        :type y:
        
        """
        
        return (output_activations-y)

    
    
#### Miscellaneous functions
def sigmoid(z):
    """
    The sigmoid function
    """
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """
    Derivative of the sigmoid function
    """
    return sigmoid(z)*(1-sigmoid(z))


In [35]:
training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8987 / 10000
Epoch 1: 9160 / 10000
Epoch 2: 9301 / 10000
Epoch 3: 9324 / 10000
Epoch 4: 9346 / 10000
Epoch 5: 9362 / 10000
Epoch 6: 9375 / 10000
Epoch 7: 9370 / 10000
Epoch 8: 9378 / 10000
Epoch 9: 9418 / 10000
Epoch 10: 9408 / 10000
Epoch 11: 9427 / 10000
Epoch 12: 9417 / 10000
Epoch 13: 9459 / 10000
Epoch 14: 9434 / 10000
Epoch 15: 9457 / 10000
Epoch 16: 9457 / 10000
Epoch 17: 9449 / 10000
Epoch 18: 9445 / 10000
Epoch 19: 9454 / 10000
Epoch 20: 9451 / 10000
Epoch 21: 9454 / 10000
Epoch 22: 9452 / 10000
Epoch 23: 9471 / 10000
Epoch 24: 9453 / 10000
Epoch 25: 9468 / 10000
Epoch 26: 9467 / 10000
Epoch 27: 9488 / 10000
Epoch 28: 9448 / 10000
Epoch 29: 9485 / 10000
